In [1]:
from typing import Dict, Iterable, Optional

import numpy as np
import torch
from torch.distributions import Normal, Poisson
from torch.distributions import kl_divergence as kld
from torch import tensor
from complementary_models import HALOVIR as HALOVI
from complementary_models import HALOVAER as HALOVAE
import scanpy as sc
import scvi
import pandas as pd
# torch.autograd.set_detect_anomaly(True)

Global seed set to 0


In [2]:
### test whole data with RNA only 
# adata_multi = sc.read_h5ad("halo/E18_mouse_Brain/multiomic.h5ad")
# adata_multi.obs["batch_id"] = 1
# adata_multi.var["modality"] =adata_multi.var["feature_types"]
# adata_mvi = scvi.data.organize_multiome_anndatas(adata_multi)


# df_meta= pd.read_csv("halo/E18_mouse_Brain/RNA/metadata.tsv",sep = "\t",index_col=0)
# bins = df_meta.binned.unique()
# times = {}
# index = 0
# for bin in sorted(bins):
#     times[bin] = index
#     index += 1

# def add_time(row, times):
#     timestamp = times[row.binned]
#     return timestamp

# df_meta['time_key'] = df_meta.apply(lambda row: add_time(row, times), axis=1)

# newindex = []

# for idx, row in df_meta.iterrows():
#     newindex.append(idx+"_paired")

# df_meta['Id'] = newindex    

# df_meta_sub = df_meta[["Id", 'latent_time']]

# df_meta_sub.set_index("Id", inplace=True)
# adata_mvi.obs = adata_mvi.obs.join(df_meta_sub, how="inner")
# sc.pp.filter_genes(adata_mvi, min_cells=int(adata_mvi.shape[0] * 0.01))



adata_multi = sc.read_h5ad("openproblem/neurips_traj.h5ad")
adata_multi.X = adata_multi.layers['counts']
newadata = adata_multi[adata_multi.obs['GEX_pseudotime_order'].notna()].copy()
adata_mvi = scvi.data.organize_multiome_anndatas(newadata)

adata_mvi 

AnnData object with n_obs × n_vars = 10862 × 129921
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'modality'
    var: 'feature_types', 'gene_id'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap'
    layers: 'counts'

In [4]:
HALOVI.setup_anndata(adata_mvi, batch_key="batch", time_key='GEX_pseudotime_order')
model = HALOVI(
    adata_mvi,
    n_genes=(adata_mvi.var['feature_types']=='GEX').sum(),
    n_regions=(adata_mvi.var['feature_types']=='ATAC').sum()
)

In [5]:
model.module.set_train_params(expr_train=True, acc_train=True)
model.module.set_finetune_params(0)
model.train(max_epochs=10)

/home/haiyi/anaconda3/envs/halo/lib/python3.9/site-packages/scvi/model/base/_training_mixin.py:67: UserWarning: max_epochs=10 is less than n_epochs_kl_warmup=400. The max_kl_weight will not be reached during training.
  warnings.warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 10/10: 100%|██████████| 10/10 [01:17<00:00,  7.72s/it, loss=4.64e+04, v_num=1]


In [6]:
model.module.set_train_params(expr_train=True, acc_train=False)
model.module.set_finetune_params(0)
model.train(max_epochs=10)

/home/haiyi/anaconda3/envs/halo/lib/python3.9/site-packages/scvi/model/base/_training_mixin.py:67: UserWarning: max_epochs=10 is less than n_epochs_kl_warmup=400. The max_kl_weight will not be reached during training.
  warnings.warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 10/10: 100%|██████████| 10/10 [01:12<00:00,  7.28s/it, loss=4.11e+03, v_num=1]


In [7]:
model.module.set_train_params(expr_train=False, acc_train=True)
model.module.set_finetune_params(0)
model.train(max_epochs=10)

/home/haiyi/anaconda3/envs/halo/lib/python3.9/site-packages/scvi/model/base/_training_mixin.py:67: UserWarning: max_epochs=10 is less than n_epochs_kl_warmup=400. The max_kl_weight will not be reached during training.
  warnings.warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 10/10: 100%|██████████| 10/10 [01:11<00:00,  7.11s/it, loss=1.12e+04, v_num=1]


In [8]:
model.module.set_train_params(expr_train=True, acc_train=True)
model.module.set_finetune_params(2)
model.train(max_epochs=100)

/home/haiyi/anaconda3/envs/halo/lib/python3.9/site-packages/scvi/model/base/_training_mixin.py:67: UserWarning: max_epochs=100 is less than n_epochs_kl_warmup=400. The max_kl_weight will not be reached during training.
  warnings.warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 14/100:  13%|█▎        | 13/100 [02:16<15:06, 10.42s/it, loss=1.45e+05, v_num=1]

/home/haiyi/anaconda3/envs/halo/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [9]:
model.module.set_train_params(expr_train=True, acc_train=True)
model.module.set_finetune_params(0)
model.train(max_epochs=100)

/home/haiyi/anaconda3/envs/halo/lib/python3.9/site-packages/scvi/model/base/_training_mixin.py:67: UserWarning: max_epochs=100 is less than n_epochs_kl_warmup=400. The max_kl_weight will not be reached during training.
  warnings.warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 100/100: 100%|██████████| 100/100 [12:57<00:00,  7.78s/it, loss=4.54e+04, v_num=1]


torch.cat(latent_expr).numpy(), torch.cat(latent_atac).numpy(), \
            torch.cat(latent_expr_dep).numpy(), torch.cat(latent_atac_dep).numpy(), \
                torch.cat(latent_expr_indep).numpy(), torch.cat(latent_atac_indep).numpy(), torch.cat(times).numpy()
    

In [10]:
latent_expr, latent_acc,latent_expr_dep, latent_atac_dep, latent_expr_indep, latent_atac_indep, times  = model.get_latent_representation()

In [11]:
latent_atac_indep.shape
times.shape

(10862, 1)

In [13]:
adata_mvi

AnnData object with n_obs × n_vars = 10862 × 129921
    obs: 'GEX_pct_counts_mt', 'GEX_n_counts', 'GEX_n_genes', 'GEX_size_factors', 'GEX_phase', 'ATAC_nCount_peaks', 'ATAC_atac_fragments', 'ATAC_reads_in_peaks_frac', 'ATAC_blacklist_fraction', 'ATAC_nucleosome_signal', 'cell_type', 'batch', 'ATAC_pseudotime_order', 'GEX_pseudotime_order', 'Samplename', 'Site', 'DonorNumber', 'Modality', 'VendorLot', 'DonorID', 'DonorAge', 'DonorBMI', 'DonorBloodType', 'DonorRace', 'Ethnicity', 'DonorGender', 'QCMeds', 'DonorSmoker', 'modality', '_scvi_batch', '_scvi_labels', 'leiden_latent', 'leiden_latent_dep', 'leiden_latent_indep'
    var: 'feature_types', 'gene_id'
    uns: 'ATAC_gene_activity_var_names', 'dataset_id', 'genome', 'organism', '_scvi_uuid', '_scvi_manager_uuid', 'neighbors', 'leiden'
    obsm: 'ATAC_gene_activity', 'ATAC_lsi_full', 'ATAC_lsi_red', 'ATAC_umap', 'GEX_X_pca', 'GEX_X_umap', 'latent_rep', 'latent_rep_dep', 'latent_rep_indep'
    layers: 'counts'
    obsp: 'distances', 'co

In [11]:
## benchmark clustering 
from sklearn.metrics.cluster import adjusted_rand_score as ARI
from sklearn.metrics import normalized_mutual_info_score as NMI


latent_rep = np.concatenate((latent_expr, latent_acc), axis=1)
adata_mvi.obsm["latent_rep"] = latent_rep
sc.pp.neighbors(adata_mvi, use_rep="latent_rep")
sc.tl.leiden(adata_mvi, key_added="leiden_latent", resolution=0.4)

latent_rep_dep = np.concatenate((latent_expr_dep, latent_atac_dep), axis=1)
latent_rep_dep.shape
adata_mvi.obsm["latent_rep_dep"] = latent_rep_dep
sc.pp.neighbors(adata_mvi, use_rep="latent_rep_dep")
sc.tl.leiden(adata_mvi, key_added="leiden_latent_dep", resolution=0.4)

latent_rep_indep = np.concatenate((latent_expr_indep, latent_atac_indep), axis=1)
adata_mvi.obsm["latent_rep_indep"] = latent_rep_indep
sc.pp.neighbors(adata_mvi, use_rep="latent_rep_indep")
sc.tl.leiden(adata_mvi, key_added="leiden_latent_indep", resolution=0.4)


Epoch 14/100:  13%|█▎        | 13/100 [15:50<1:46:03, 73.14s/it, loss=1.45e+05, v_num=1]


In [14]:
ari_whole = ARI(adata_mvi.obs['cell_type'], adata_mvi.obs['leiden_latent'])
nmi_whole = NMI(adata_mvi.obs['cell_type'], adata_mvi.obs['leiden_latent'])


nmi_dep = NMI(adata_mvi.obs['cell_type'], adata_mvi.obs['leiden_latent_dep'])
ari_dep = ARI(adata_mvi.obs['cell_type'], adata_mvi.obs['leiden_latent_dep'])


nmi_indep = NMI(adata_mvi.obs['cell_type'], adata_mvi.obs['leiden_latent_indep'])
ari_indep = ARI(adata_mvi.obs['cell_type'], adata_mvi.obs['leiden_latent_indep'])

print("ari_whole {}, nmi_whole: {}".format(ari_whole, nmi_whole))
print("ari_dep {}, nmi_dep: {}".format(ari_dep, nmi_dep))
print("ari_indep {}, nmi_indep: {}".format(ari_indep, nmi_indep))

ari_whole 0.4430810226476561, nmi_whole: 0.5312440155535496
ari_dep 0.3896577201655209, nmi_dep: 0.48522298385494855
ari_indep 0.44435354624353146, nmi_indep: 0.5357642472369234


In [12]:
from complementary_models import torch_infer_nonsta_dir


latent_atac_indep = torch.tensor(latent_atac_indep[:5000, :]).to('cuda')
latent_expr_indep = torch.tensor(latent_expr_indep[:5000, :]).to('cuda')
times = torch.tensor(times[:5000, :]).to('cuda')

score3, _, _ = torch_infer_nonsta_dir(latent_atac_indep, latent_expr_indep, times)
score4, _, _ = torch_infer_nonsta_dir(latent_expr_indep, latent_atac_indep, times)
print("score3 {} and score4 {}".format(score3, score4))
score3 - score4

score3 0.05627993930527525 and score4 0.0642378831053579


tensor(-0.0080, device='cuda:0', dtype=torch.float64)

In [13]:
latent_atac_dep = torch.tensor(latent_atac_dep[:5000, :]).to('cuda')
latent_expr_dep = torch.tensor(latent_expr_dep[:5000, :]).to('cuda')
# times = torch.tensor(times).to('cuda')

score1, _, _ = torch_infer_nonsta_dir(latent_atac_dep, latent_expr_dep, times)
score2, _, _ = torch_infer_nonsta_dir(latent_expr_dep, latent_atac_dep, times)
print("score1 {} and score2 {}".format(score1, score2))
score1 - score2

score1 0.05464940435763029 and score2 0.05465027092730652


tensor(-8.6657e-07, device='cuda:0', dtype=torch.float64)

In [15]:
#### MultiVI and GLUE test

HALOVI.setup_anndata(adata_mvi, batch_key="batch", time_key='GEX_pseudotime_order')
model2 = HALOVI(
    adata_mvi,
    n_genes=(adata_mvi.var['feature_types']=='GEX').sum(),
    n_regions=(adata_mvi.var['feature_types']=='ATAC').sum()
)

In [16]:
model2.module.set_train_params(expr_train=True, acc_train=True)
model.module.set_finetune_params(0)
model2.train(max_epochs=20)

/home/haiyi/anaconda3/envs/halo/lib/python3.9/site-packages/scvi/model/base/_training_mixin.py:67: UserWarning: max_epochs=20 is less than n_epochs_kl_warmup=400. The max_kl_weight will not be reached during training.
  warnings.warn(
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 20/20: 100%|██████████| 20/20 [02:39<00:00,  7.96s/it, loss=4.68e+04, v_num=1]


In [17]:
latent_expr, latent_acc,latent_expr_dep, latent_atac_dep, latent_expr_indep, latent_atac_indep, times  = model2.get_latent_representation()

In [17]:
z_multi_VI = 1/2 * (latent_expr[:5000, :]+ latent_acc[:5000, :])
z_multi_VI.shape

(5000, 10)

In [19]:
## evaluate clustering

from sklearn.metrics.cluster import adjusted_rand_score as ARI
from sklearn.metrics import normalized_mutual_info_score as NMI


latent_rep = np.concatenate((latent_expr, latent_acc), axis=1)
adata_mvi.obsm["latent_rep"] = latent_rep
sc.pp.neighbors(adata_mvi, use_rep="latent_rep")
sc.tl.leiden(adata_mvi, key_added="leiden_latent", resolution=0.4)
ari_score = ARI(adata_mvi.obs['cell_type'], adata_mvi.obs['leiden_latent'])
nmi_whole = NMI(adata_mvi.obs['cell_type'], adata_mvi.obs['leiden_latent'])

print("ARI {}, NMI {}".format(ari_score, nmi_whole))

ARI 0.33799443287137076, NMI 0.5034644361297838


In [20]:
z_multi_VI = 1/2 * (latent_expr+ latent_acc)
adata_mvi.obsm["latent_rep"] = z_multi_VI
sc.pp.neighbors(adata_mvi, use_rep="latent_rep")
sc.tl.leiden(adata_mvi, key_added="leiden_latent", resolution=0.4)
ari_score = ARI(adata_mvi.obs['cell_type'], adata_mvi.obs['leiden_latent'])
nmi_whole = NMI(adata_mvi.obs['cell_type'], adata_mvi.obs['leiden_latent'])

print("ARI {}, NMI {}".format(ari_score, nmi_whole))

ARI 0.3243323806046435, NMI 0.496832264818793


In [22]:
gene_expr = adata_mvi.X[:, :13431]
gene_peak = adata_mvi.X[:, 13431:]

Gene_PCA = sc.pp.pca(gene_expr, n_comps=10)

adata_mvi.obsm["latent_rep"] = Gene_PCA
sc.pp.neighbors(adata_mvi, use_rep="latent_rep")
sc.tl.leiden(adata_mvi, key_added="leiden_latent", resolution=0.4)
ari_score = ARI(adata_mvi.obs['cell_type'], adata_mvi.obs['leiden_latent'])
nmi_whole = NMI(adata_mvi.obs['cell_type'], adata_mvi.obs['leiden_latent'])
print("ARI {}, NMI {}".format(ari_score, nmi_whole))

Peak_SVD = sc.pp.pca(gene_peak, n_comps=10, svd_solver='arpack')
adata_mvi.obsm["latent_rep"] = Peak_SVD
sc.pp.neighbors(adata_mvi, use_rep="latent_rep")
sc.tl.leiden(adata_mvi, key_added="leiden_latent", resolution=0.4)
ari_score = ARI(adata_mvi.obs['cell_type'], adata_mvi.obs['leiden_latent'])
nmi_whole = NMI(adata_mvi.obs['cell_type'], adata_mvi.obs['leiden_latent'])
print("ARI {}, NMI {}".format(ari_score, nmi_whole))

ARI 0.1379728188454027, NMI 0.32919537383442593
ARI 0.14783618451466504, NMI 0.2825659254404057


In [19]:
from complementary_models import torch_infer_nonsta_dir

length=z_multi_VI.shape[1]
scores = []
z_multi_acc = z_multi_VI[: ,:5]
z_multi_expr = z_multi_VI[: ,5:]

z_multi_acc = torch.tensor(z_multi_acc).to('cuda')
z_multi_expr = torch.tensor(z_multi_expr).to('cuda')
times = torch.tensor(times[:5000, :]).to('cuda')
score1, _, _ = torch_infer_nonsta_dir(z_multi_acc, z_multi_expr, times)
score2, _, _ = torch_infer_nonsta_dir(z_multi_expr, z_multi_acc, times)

# for i in range(0, length, 2):
#     z1 =  torch.tensor(z_multi_VI[:, i]).to('cuda')
#     z2 = torch.tensor(z_multi_VI[:, i+1]).to('cuda')
#     print(z1.shape, z2.shape)
#     score, _, _ = torch_infer_nonsta_dir(z1, z2, times)
#     scores.append(score)

print("score1 {} and score2 {}".format(score1, score2))
score1 - score2

/tmp/ipykernel_20139/3730558050.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  times = torch.tensor(times[:5000, :]).to('cuda')


score1 0.05511208374970013 and score2 0.05498806455610048


tensor(0.0001, device='cuda:0', dtype=torch.float64)

In [20]:
glue_acc =  torch.tensor(latent_acc[:5000, :]).to('cuda')
glue_exp = torch.tensor(latent_expr[:5000, :]).to('cuda')
# times = torch.tensor(times).to('cuda')

score1, _, _ = torch_infer_nonsta_dir(glue_acc, glue_exp, times)
score2, _, _ = torch_infer_nonsta_dir(glue_acc, glue_exp, times)

print("score1 {} and score2 {}".format(score1, score2))
score1 - score2

score1 0.05500114815794869 and score2 0.05500114815794869


tensor(0., device='cuda:0', dtype=torch.float64)

In [21]:
print((adata_mvi.var["feature_types"]=="GEX").sum())
print((adata_mvi.var["feature_types"]!="GEX").sum())

13431
116490


In [22]:
gene_expr = adata_mvi.X[:, :13431]
gene_peak = adata_mvi.X[:, 13431:]
print(gene_expr.shape, gene_peak.shape)

(10862, 13431) (10862, 116490)


In [23]:
Gene_PCA = sc.pp.pca(gene_expr, n_comps=10)
Peak_SVD = sc.pp.pca(gene_expr, n_comps=10, svd_solver='arpack')

In [24]:
Peak_SVD =  torch.tensor(Peak_SVD[:5000, :]).to('cuda')
Gene_PCA = torch.tensor(Gene_PCA[:5000, :]).to('cuda')
# times = torch.tensor(times).to('cuda')

score1, _, _ = torch_infer_nonsta_dir(Peak_SVD, Gene_PCA, times)
score2, _, _ = torch_infer_nonsta_dir(Gene_PCA, Peak_SVD, times)

print("score1 {} and score2 {}".format(score1, score2))
score1 - score2

score1 0.0548628958826182 and score2 0.05487209351550488


tensor(-9.1976e-06, device='cuda:0', dtype=torch.float64)